# 1. Establish communication with the card

## 1.1. Find the smartcard reader

In [64]:
from applets.helpers import get_connection, AppletBase

In [65]:
connection = get_connection()
if connection is None:
    print("Failed to open a connection. No cardreaders?")

## 1.2. Power on the card and get `ATR`
`ATR` - Answer To Reset, bytes retured by the card when it is powered on.<br>
It helps to determine communication protocol and frequency we should choose.

In [66]:
atr=bytes(connection.getATR())
print(atr.hex())

3bdc18ff8191fe1fc38073c821136605036351000250


## 1.3. Select the applet by AID
JavaCard can have multiple applets installed. They are like very isolated "apps".<br>
They can't talk to each other and only one can be active at any time.

In [67]:
from applets.teapot import Teapot

class MemoryCard(Teapot):
    def __init__(self, connection=None):
        super().__init__(connection)
        self.AID = "B00B5111CB01"

    def get_random(self):
        return self.request("B0B10000")

In [68]:
app = MemoryCard(connection)
app.select()

b''

In [69]:
app.get_random().hex()

'72a79b5445132c88e918cc6574427d1231b18ef0f7cc90953441662e1a3bf378'

# 2. Establish secure communication

At the moment we use a homebrew version, I need to understand and implement Noise in the near future.

[Noise protocol](https://noiseprotocol.org/noise.html) should be with the following primitives:
- secp256k1 as signature scheme
- AES CBC with random IV as a symmetric cipher
- SHA256 as a hash function, HMAC_SHA256 as authentication code

AES CBC because we have hardware implementation on the card, so we can use the Key class that is more secure than byte array. HMAC sucks here though...

Generally a shared secret is split to 2 different secrets for encryption and decryption. This allows us to use AES in counter mode where the counter is the same for both host and the card - they use different keys so it's ok.

Description of the current protocol:
1. Get the static public key from the card. If it is already known - proceed with step 2
2. Generate a random key pair, calculate a shared secret `s = SHA512(ECDH(d,P))` and split it to two symmetric keys - `enc = s[:32]` for encryption and `dec = s[32:]` for decryption (second 32 bytes)
3. Send your pubkey from keypair in step 2 as associated data to the card: `<data:pub_uncompressed><hmac(enc,data>`
4. Card will return its fresh public key encrypted: `<ciphertext:iv|aes(dec,iv,pub_card)><hmac(dec,ciphertext)>`
5. Verify the `hmac`, decrypt the `pub_card=decrypt(dec,ct)`
6. Calculate new shared secret `s=SHA512(ECDH(d,pub_card))`, set counter to zero `iv=0`, use `dec = s[32:]` for decryption, `enc = s[32:]` for enctyption. Increase the counter on every request, reestablish communication if the counter reaches max value (16 bytes, shouldn't happen in most cases)
7. All secure messages now should be in the form `<ct:aes(enc,iv,command|data)><hmac_sha256(enc,ct)>`, `iv` is not transmitted but increased for every new command. Returned data will be of the same form but using the `dec` key. `iv` will be the same.

Maximum size of the APDU is 256 bytes. Maximum size of the mnemonic is `9*24-1=215` bytes plus length and command and rounding to `16` bytes, plus `32`-byte hmac - we are at `256` bytes... We can send at max `255` bytes.

So to store and retrieve large mnemonic we need to split it to two pieces, and provide a function to set part of the data array. Therefore for secret storage we have to make an API: 
- `set_data<data>` / `get_data` - for short mnemonics
- `set_data_length<len>` - should be called first, erases existing data, 
- `set_data_part <start><data>` - stores part of the data with an offset
- `get_data_length`
- `get_data_part <start><len>` - gets part of the data from `start` with `len`


## 2.1. Get public key from the card

In [70]:
# embit library: https://github.com/diybitcoinhardware/embit
# works with python3 and micropython
from embit.ec import PrivateKey, PublicKey, secp256k1

In [71]:
# get pubkey - private key is generated when applet is installed
sec = app.request("B0B2000000")
card_pubkey = PublicKey.parse(sec)
card_pubkey

PublicKey(045fdd66bff43c2e0a4c85099729a7154e767e0416a9f20f2fb6d3f1cf4bdeaf0c8f633c493dac541a3be2a5ac49ae009f19a1176d401d00c1c1d67d935c14aaa1)

## 2.2. Select a random secret on the host

In [72]:
import os
import hashlib
secret = os.urandom(32)
# ugly copy...
pub = PublicKey.parse(card_pubkey.sec())
secp256k1.ec_pubkey_tweak_mul(pub._point, secret)
shared_secret = pub.sec()[1:33]
shared_hash = hashlib.sha256(shared_secret).digest()
print("Shared secret:", shared_secret.hex())
print("Sha256 of the secret:", shared_hash.hex())

Shared secret: 32d8f1510d536bfdf120261c0609325c31f3ecbe465db9c5513c53bd4631621a
Sha256 of the secret: bee15708c2d76f11c7243175a8be2cec63b73c68ac40fe9cbfb3ff4a4c1634dd


In [73]:
host_prv = PrivateKey(secret)
host_pub = host_prv.get_public_key()
host_pub.compressed = False
data = bytes([65])+host_pub.sec()
data.hex()

'410403f60c7cd532a851db6529544608037be95ca8674a0dfb56ffa795235005d785c17ba5f24d92dbffac79e6cacf6b44bfdd62de69951506afb87d193f1c3c38b1'

In [74]:
res = app.request("B0B30000"+data.hex())
if res == shared_hash:
    print("Yey! We have a shared secret now!")
else:
    print("Meh... Something didn't work? Card returned %s, we have %s" % (res.hex(), shared_hash.hex()))

Yey! We have a shared secret now!


In [75]:
# get random data
rand = app.request("B0B2000000")
rand.hex()

'045fdd66bff43c2e0a4c85099729a7154e767e0416a9f20f2fb6d3f1cf4bdeaf0c8f633c493dac541a3be2a5ac49ae009f19a1176d401d00c1c1d67d935c14aaa1'

In [76]:
# static data for any other APDU
data = app.request("B0A1000000")
data

b'I am a teapot gimme some tea plz'

In [77]:
(bytes([len(data)])+data).hex()

'204920616d206120746561706f742067696d6d6520736f6d652074656120706c7a'

In [78]:
# static data for any other APDU
c = app.request("B0A40000"+(bytes([len(data)])+data).hex())
c

b'TnQ\x93\xa9L=\xba\x85d\xbf\x11\xc1C\xc9,/-54=\xa0\x9b\x8e\\)\xe6\xc8u\xc9q\x0e\x8e:+\xd7\x9d\xfdp\xa2\x1c\x9e\xa8\xf1\x14\x13\x92\x91\xe49\x93M\x08.\x16\xf3],\x91\xeb\x80\xeeToi0\x956&\xbb\xe3oG\xab\xed\x06\xc9\xf3N\xac_\xf5\xd5G\x1d\x1a\x98n!\xdc\xd6\xa3\xdf\xbc8\x1c'

In [79]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import hmac

def decrypt(c,secret):
    recv_hmac = c[-32:]
    iv = c[:16]
    ct = c[16:-32]
    h = hmac.new(shared_secret, digestmod='sha256')
    h.update(iv)
    h.update(ct)
    expected_hmac = h.digest()
    if expected_hmac != recv_hmac:
        raise RuntimeError("Wrong HMAC. Got %s, expected %s"%(recv_hmac.hex(),expected_hmac.hex()))
    backend = default_backend()
    cipher = Cipher(algorithms.AES(secret), modes.CBC(iv), backend=backend)
    decryptor = cipher.decryptor()
    # remove \x80... padding
    plain = decryptor.update(ct)+decryptor.finalize()
    return b"".join(plain.split(b"\x80")[:-1])

decrypt(c, shared_secret)

b'I am a teapot gimme some tea plz'

In [80]:
connection.disconnect()